In [ ]:
%pip install langchain openai cohere tiktoken

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/65/f4/dea10c11f0c2ebca298ba57f546940f8944e49c624239183c924ae0e1f81/langchain-0.0.339-py3-none-any.whl.metadata
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/30/1d/27c3571504fb6fb1e9f7c906d93590ead22f5f34910489e155ee28512eeb/openai-1.3.5-py3-none-any.whl.metadata
  Obtaining dependency information for cohere from https://files.pythonhosted.org/packages/6c/b9/67b691fc5a8fa208435f85b38671d36238ab81eb1e7190c7c1e3b34892c7/cohere-4.36-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/94/2f/0cc8fb3436d421d8fa2da370aca0283201f1b99e88a0f6e742bd8eef397d/tiktoken-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/8d/e2/528c52001a743a7faa28e6d3095d9f01b472d3ef

In [2]:
import json
import random

example_file = open("../json/example.json","r")
test_file = open("../json/test.json","r")
format = open("../json/format.json","r").read()
tools = open("../json/tools.json","r").read()


In [3]:
# get examples
test_examples = json.loads(test_file.read())
examples = json.loads(example_file.read())

In [4]:
prompt_template= """
{tools_template}

{format_template}

{example_template}

{start_template}
"""

tools_template = """
Use the following tools and answer the given question using given answer format, 
{tools}
"""
answer_format_template = """
Generated answer for any given question should have following format
{format}
"""

example_template = """
Following are the example of question-answer pairs for the abovementioned tools
Example 1
{example1}

Example 2
{example2}
"""

start_template = """
To reference the value of the ith tool in the chain, use $$PREV[i] as argument value. i = 0, 1, .. j-1; j = current tool’s index in the array
If the query could not be answered with the given set of tools, output an empty list instead.

Empty list should look like []. Whenever you want to access information about current user to pass it further, you need to make a call to whoami api tool.

Answer the following question based on instructions provided above.
question: {question}
answer:{{
"question":
"answer":
}}"""



In [5]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

full_prompt = PromptTemplate.from_template(prompt_template)
tools_prompt = PromptTemplate.from_template(tools_template)
answer_format_prompt = PromptTemplate.from_template(answer_format_template)
example_prompt = PromptTemplate.from_template(example_template)
start_prompt = PromptTemplate.from_template(start_template)

input_prompt = [
    ("tools_template",tools_prompt),
    ("format_template",answer_format_prompt),
    ("example_template",example_prompt),
    ("start_template",start_prompt)
]
prompt = PipelinePromptTemplate(final_prompt=full_prompt,pipeline_prompts=input_prompt)
prompt.input_variables

['example1', 'format', 'example2', 'tools', 'question']

In [7]:
file = open("../prompt/zero_shot_with_two_example.txt","w")
file.write(prompt.format(question="",tools=tools,format=format,example1=examples[0],example2=examples[1]))
file.close()

In [ ]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
config = open("config.txt","r").read()
OPENAI_API_KEY=config
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp

llm = LlamaCpp(model_path='../models/llama-2-7b-32k-instruct.Q8_0.gguf',n_ctx=4096)
# chain = LLMChain(llm = ChatOpenAI(model='gpt-4'), prompt=prompt)
chain = LLMChain(llm = llm, prompt=prompt)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../models/llama-2-7b-32k-instruct.Q8_0.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q8_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q8_0     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_down.weight q8_0     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    7:              blk.0.ffn_up.weight q8_0     [  4096, 11008,     1, 

In [46]:
response = []
for test in test_examples:

    print(test['question_id'])
    # input = prompt.format(question=test['question'],tools=tools,format=format,examples=examples)
    output = chain.invoke({'question':test['question'],'tools':tools,'format':format,'example1':examples[0],'example2':examples[1]})

    answer = output['text']
    if (answer.find('{') < 0):
        answer = []
    else:
        try:
            answer = json.loads(output['text'])
        except:
            answer = json.loads(answer[answer.find("{"):])
    if (answer == []):
        answer = {'question':test['question'],'answer':[]}
    answer['question_id'] = test['question_id']
    response.append(answer)
    
    if (test['question_id'] == '3'):
        break


2
3



llama_print_timings:        load time =     227.55 ms
llama_print_timings:      sample time =      85.17 ms /   256 runs   (    0.33 ms per token,  3005.79 tokens per second)
llama_print_timings: prompt eval time =   73315.60 ms /  2223 tokens (   32.98 ms per token,    30.32 tokens per second)
llama_print_timings:        eval time =   38622.20 ms /   255 runs   (  151.46 ms per token,     6.60 tokens per second)
llama_print_timings:       total time =  113283.66 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =     227.55 ms
llama_print_timings:      sample time =      85.40 ms /   256 runs   (    0.33 ms per token,  2997.76 tokens per second)
llama_print_timings: prompt eval time =    1073.90 ms /    29 tokens (   37.03 ms per token,    27.00 tokens per second)
llama_print_timings:        eval time =   37977.28 ms /   255 runs   (  148.93 ms per token,     6.71 tokens per second)
llama_print_timings:       total time =   39835.63 ms


In [30]:
output

{'question': 'List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them. ',
 'tools': '[\n    {\n        "tool_name":"works_list",\n        "tool_description":"Returns a list of work items matching the request",\n        "arguments":[\n            {\n                "name":"applies_to_part",\n                "description":"Filters for work belonging to any of the provided parts",\n                "type":"array of strings",\n                "example":["FEAT-123",\n                "ENH-123",\n                "PROD-123",\n                "CAPL-123"]\n            },\n            {\n                "name":"created_by",\n                "description":"Filters for work created by any of these users",\n                "type":"array of strings",\n                "example":["DEVU-123"]\n            },\n            {\n                "name":"issue.priority",\n                "description":"Filters for issues with any of the provided priorities. Allow

In [24]:
output

{'question': 'What is the meaning of life?',
 'tools': '[\n    {\n        "tool_name":"works_list",\n        "tool_description":"Returns a list of work items matching the request",\n        "arguments":[\n            {\n                "name":"applies_to_part",\n                "description":"Filters for work belonging to any of the provided parts",\n                "type":"array of strings",\n                "example":["FEAT-123",\n                "ENH-123",\n                "PROD-123",\n                "CAPL-123"]\n            },\n            {\n                "name":"created_by",\n                "description":"Filters for work created by any of these users",\n                "type":"array of strings",\n                "example":["DEVU-123"]\n            },\n            {\n                "name":"issue.priority",\n                "description":"Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2, p3",\n                "type":"array of strings",\n     

In [44]:
outfile = open("../output/zero shot with two examples/llama_7B_instruct_res.json","w")
json.dump(response,outfile,indent=4)
outfile.close()